# Variables

## Import libraries

In [34]:
import os
import sys
from rdflib import URIRef, Literal

## Defining global variables

In [35]:
# Existing files
ont_file_name = "ontology.ttl"
ruleset_file_name = "rules.pie"

# Created files during process (in `tmp_folder`)
export_file_name = "addresses-temp.ttl"
out_file_name = "addresses.ttl"
local_config_file_name = "config_repo.ttl"
facts_ttl_file_name = "facts_data.ttl"
implicit_to_facts_ttl_file_name = "implicit_to_facts.ttl"

# Existing folders
data_folder_name = "../data"

# Created folder during process
tmp_folder_name = "../tmp_files"

# GraphDB repository name
facts_repository_name = "addresses_from_factoids"

# Definition of names of named graphes 
ontology_named_graph_name = "ontology"
facts_named_graph_name = "facts"
factoids_named_graph_name = "factoids"
permanent_named_graph_name = "permanent"
tmp_named_graph_name = "temporary"
inter_sources_name_graph_name = "inter_sources"

# URIs to access to GraphDB
str_graphdb_url = "http://localhost:7200"

py_code_folder_path = "./code"

## Processing global variables

* Obtaining absolute file paths from the relative paths given in the previous section
* Create a temporary folder if it doesn't already exist to store files to be deleted.
* Get RDFLib object for `graphdb_url` from a string

In [36]:
tmp_folder = os.path.abspath(tmp_folder_name)
data_folder = os.path.abspath(data_folder_name)

python_code_folder = os.path.abspath(py_code_folder_path)

local_config_file = os.path.join(tmp_folder, local_config_file_name)
ont_file = os.path.abspath(ont_file_name)
ruleset_file = os.path.abspath(ruleset_file_name)
facts_ttl_file = os.path.join(tmp_folder, facts_ttl_file_name)
implicit_to_facts_ttl_file = os.path.join(tmp_folder, implicit_to_facts_ttl_file_name)

graphdb_url = URIRef(str_graphdb_url)

## Import modules located in `code` folder

In [37]:
# Calling up the `code` folder contains the python codes
sys.path.insert(1, python_code_folder)

import filemanagement as fm
import graphdb as gd
import graphrdf as gr
import attributeversioncomparisons as avc
import multisourcesprocessing as msp
import factoidscreation as fc
import timeprocessing as tp
import wikidata as wd

## Select streets of Paris with their French Wikipedia page

In [38]:
# Sélection des voies de Paris avec une page Wikidata
query = """
SELECT DISTINCT ?voie ?page WHERE {
  {?voie p:P361 [ps:P361 wd:Q16024163] } UNION {?voie p:P361 [ps:P361 wd:Q107311481] }
  ?page schema:about ?voie ; schema:isPartOf <https://fr.wikipedia.org/> .
}
"""
res = wd.get_select_query_wikidata(query)

In [64]:
import requests
import urllib.parse as up

def read_wikipedia_page(page_name, lang):
    # URL de l'API Wikipedia
    url = f"https://{lang}.wikipedia.org/w/api.php"

    # Paramètres de la requête
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "titles": page_name,
        "explaintext": True,
    }

    # Effectuer la requête
    response = requests.get(url, params=params)

    # Vérifier si la requête a réussi
    if response.status_code == 200:
        data = response.json()
        pages = data.get("query", {}).get("pages", {})
        # print(pages)
        for page_id, page_value in pages.items():
            page_content = page_value.get("extract", "")
    else:
        print("Erreur lors de la requête :", response.status_code)
        page_content = None

    return page_content, response.status_code

In [68]:
import random

res2 = wd.construct_table_results_from_json(res)

page_content_list = []
fr_wiki_uri = "https://fr.wikipedia.org/wiki/"
nb_pages = 10
for i in range(nb_pages):
    random_elem = random.choice(res2[1:])
    page_uri = random_elem[1]
    page_name = up.unquote(page_uri.replace(fr_wiki_uri, ""))
    page_content, status_code = read_wikipedia_page(page_name, "fr")
    if status_code == 200:
        page_content_list.append([page_name, page_content])

1182 Rue_Madeleine-Brès
896 Rue_de_l'Indre_(Paris)
524 Esplanade_Gilles-Jacquier
1166 Rue_Lheureux
1039 Passage_Boiton
1781 Rue_de_l'Hôtel-Saint-Paul
769 Rue_du_Général-Roques
2192 Place_Auguste-Baron
3759 Place_Beauvau
902 Allée_Maria-Callas


# Prompting interface

Requirements :
- A serving Ollama API
- Name of the model use to create the running Ollama API

You can run this notebook in Google Colab or in a personal Pyhton environnement.

In [41]:
!pip install ollama

Defaulting to user installation because normal site-packages is not writeable


## Select the model name

In [58]:
from ipywidgets import widgets
from IPython.display import display

# Define the choices and their associated text (model value is Ollama code name of the model to load).
# New choices can be added by searching models in Ollama : https://ollama.com/
# The models proposed here are small or small-to-mdedium size models that can be run in Colab.
choices = {'Llama 3.1 (8B)': 'llama3',
           'Gemma (9B)': 'gemma2:9b',
           'Mistral Small 2 (22B)':'mistral-small',
           'Phi 3.5 (3.8B)':'phi3.5:3.8b'}

# Create a dropdown widget
dropdown = widgets.Dropdown(options=choices.keys())

# Display the dropdown widget
display(dropdown)

# Define a function that updates the MODEL variable based on the user's selection
def update_model(change):
    global MODEL
    MODEL = choices[change.new]

# Attach the function to the dropdown widget
dropdown.observe(update_model, names='value')

# Initialize the MODEL variable
MODEL = choices[dropdown.value]

Dropdown(options=('Llama 3.1 (8B)', 'Gemma (9B)', 'Mistral Small 2 (22B)', 'Phi 3.5 (3.8B)'), value='Llama 3.1…

## Set the LLAMA HOST address
If you use Google Colab : It needs to be updated each time you launch the two notebooks.

In [43]:
HOST='https://2d79-35-240-193-54.ngrok-free.app/'

Copy HOST value printed at the end of the *00_install_ollama_and_download_model* notebook.

In [44]:
%env OLLAMA_HOST={HOST}

env: OLLAMA_HOST=https://2d79-35-240-193-54.ngrok-free.app/


If you search for *https://replace-by-your-own.ngrok-free.app* in Google and click on *Visit website*, you should see a web page with the message **Ollama is running**

## Prompting interface

In [60]:
!ollama pull "phi3.5:3.8b"
print(MODEL)

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling b5374915da53...   0% ▕                ▏    0 B/2.2 GB                  pulling manifest 
pulling b5374915da53...   0% ▕                ▏    0 B/2.2 GB                  pulling manifest 
pulling b5374915da53...   0% ▕                ▏    0 B/2.2 GB                  pulling manifest 
pulling b5374915da53...   0% ▕                ▏    0 B/2.2 GB                  pulling manifest 
pulling b5374915da53...   0% ▕                ▏    0 B/2.2 GB                  pulling manifest 
pulling b5374915da53...   0% ▕                ▏    0 B/2.2 GB                  pulling manifest 
pulling b5374915da53...   0% ▕         

In [61]:
import ollama

# Roles
USER = 'user' #you
ASSISTANT = 'assistant' #the LLM

In [ ]:
messages = []
responses = []

# Created from this code https://github.com/ollama/ollama-python/issues/242
def add_history(content, role):
    messages.append({'role': role, 'content': content})

def chat(message):
    add_history(message, USER)
    response = ollama.chat(model=MODEL, messages=messages, stream=True)
    complete_message = ''
    for line in response:
        complete_message += line['message']['content']
        print(line['message']['content'], end='', flush=True)
    responses.append(complete_message)
    add_history(complete_message, ASSISTANT)

content1 = "Salut, peux tu m'extraire dans ce wikicode l'ensemble des valeurs temporelles. Si ces valeurs sont floues, tu peux m'ajouter la précision. Par exemple, je te donne le contenu de la page de la rue Serpente : 'La rue Serpente est une voie située dans le quartier de la Monnaie dans le 6e arrondissement de Paris, en France.\n\n\n== Situation et accès ==\nLa rue Serpente est desservie par la ligne 4 à la station Odéon et la ligne 10 à la station Cluny - La Sorbonne.\n\n\n== Origine du nom ==\nLa rue Serpente, qui commençait rue de la Harpe et finissait rue Hautefeuille, fut ouverte en 1179 sous le nom de vicus Serpentus, comme l'indiquent des actes notariés[réf. nécessaire], en raison des sinuosités qu'elle décrivait. En 1180, elle est appelée « rue du Serpent » ; en 1263, vicus Tortuosus qui est ad oppositis Palatii Termanum (« rue sinueuse qui est en face du palais des Thermes ») et, vers 1295, « rue de la Serpente » ou « rue Serpente ».\nVers 1280-1300, elle est citée dans Le Dit des rues de Paris de Guillot de Paris sous la forme « rue de la Serpent ».\nLe tracé actuel de la rue a absorbé une autre rue historique du vieux Paris, connue sous le nom de « rue du Battoir-Saint-André » lorsque l'odonyme disparut. Cette rue, qui commençait rue Hautefeuille et finissait rue de l'Éperon, s'appelait en 1292 « rue Jehan-de-Fontenoy ». Elle est citée dans Le Dit des rues de Paris de Guillot de Paris sous le nom de « rue de la Plâtrière » ; en 1430, elle est appelée « Haute-Rue » dite « rue Bastouer », en 1520, « rue de la Vieille-Plâtrière », puis « rue du Battoir », du fait d'une enseigne qui s'y trouvait.\n\n\n== Historique ==\nLa rue actuelle résulte de la fusion, en 1851, de la rue du Battoir-Saint-André entre la rue Hautefeuille et la rue de l'Éperon, et la rue Serpente entre la rue de la Harpe et la rue Hautefeuille.\n\n\n=== Rue Serpente ===\nLa rue Serpente, qui commençait la rue de la Harpe et finissait rue Hautefeuille, fut ouverte en 1179, sous le nom de vicus Serpentus, comme l'indiquent des actes notariés, en raison des sinuosités qu'elle décrivait.  \nEn 1180, elle est appelée « rue du Serpent », en 1263, vicus Tortuosus qui est ad oppositis Palatii Termanum (« rue sinueuse qui est en face du palais des Thermes ») et vers 1295, « rue de la Serpente » ou « rue Serpente ».\nVers 1280-1300, elle est citée dans Le Dit des rues de Paris de Guillot de Paris sous la forme « rue de la Serpent ».\nElle est citée sous le nom de « rue de la Serpente » dans un manuscrit de 1636.\n\n\n=== Rue du Battoir-Saint-André ===\nLa rue du Battoir-Saint-André, qui commençait rue Hautefeuille et finissait rue de l'Éperon, s'appelait en 1292 « rue Jehan-de-Fontenoy ». \nElle est citée dans Le Dit des rues de Paris sous le nom de « rue de la Plâtrière ».\nEn 1430, elle est appelée « Haute-Rue » dite « rue Bastouer ».\nEn 1520, « rue de la Vieille-Plâtrière » puis « rue du Battoir » en raison d'une enseigne.\nElle est citée sous le nom de « rue du Battouer » dans un manuscrit de 1636.\n\n\n== Bâtiments remarquables et lieux de mémoire ==\nLes archives de la presse permettent de retrouver la trace et de situer plusieurs librairies ou organes de presse établis dans la rue il y a plus de deux siècles : au no 9 le bureau du Journal des Dames en 1791, au no 13 la librairie Aimé Payen en 1825, au no 14 l’éditeur-libraire Chez Plancher en 1817, au no 16 Chez L’Huillier en 1816, au no 17 Chez Garnery en 1790... Dans le roman Illusions perdues, Honoré de Balzac y établit deux de ses personnages, eux aussi libraires, qui occupent « le rez-de-chaussée d’un de ces vieux hôtels de la rue Serpente, où le cabinet de la maison se trouvait au bout de vastes salons convertis en magasins ».\nLe cinéma d'art et d'essai MK2 Hautefeuille, situé au croisement de la rue Hautefeuille.\nNo 7 : emplacement du collège de Tours, fondé par l'archevêque Étienne de Bourgueil en 1330, avant d'être rattaché à l'université de Paris en 1763.\nNo 13 : emplacement de la librairie de C.G.F. Lamyre, auquel s'associa, peu de temps avant sa mort, Déodat Gratet de Dolomieu.\nNo 14-16 : hôtel Serpente, ancien emplacement de la boutique du marchand d'estampes Henri-Louis Basan (vers 1802), fils de Pierre-François Basan, et où est installé l’éditeur-libraire Chez Plancher en 1817, puis en 1829 le libraire Gauthier frères et Cie.\nNo 15 : emplacement du collège de Suesse ou d'Upsal, fondé en 1291 ou 1315 pour les étudiants suédois de l'université de Paris par maître André, prévôt d'Uppsala en Suède, probablement pour le compte du chapitre de la cathédrale d'Uppsala. Il consiste en deux maisons adjacentes situées rue Serpente et rue des Deux-Portes. Il décline rapidement, malgré plusieurs tentatives de l'évêque et du chapitre d'Uppsala pour y envoyer des pensionnaires, et, au milieu du XIVe siècle, il est dit « que, depuis longtemps, les étudiants suédois ne viennent plus étudier à Paris[réf. nécessaire] ». En 1354, le collège est définitivement vendu à Yves Simon, secrétaire du roi de France, à charge de verser à l'avenir une pension de 7 livres parisis aux élèves suédois qui viendraient s'établir à Paris. Le jardin commun aux deux maisons reste la propriété de la nation suédoise et, à partir de 1374, est loué à un autre secrétaire royal pour la somme de 40 sous par an.\nNo 28 : la Maison de la recherche de Sorbonne Université, ancien hôtel des Sociétés savantes.\n '"
content1 += """Dans cet exemple, il existe les dates suivantes :
* 1179
* 1180
* Vers 1280-1300
* 1430
* vers 1802
* 1291 ou 1315
* à partir de 1374
"""
for elem in page_content_list:
    content1 += elem[1]
# content1 += str(page_content_list)

content2 = "Peux tu m'extraire vraiment toutes les dates qui existent"

chat(content1)
chat(content2)

print(responses)


Quelques élédictes informations sur le boulevard de Picpus, voie historique située dans les quartiers bel-aprés d'Ivry au nord du parc Montaigne. Ce boulvard est important pour son rôle durant la Première Guerre mondiale où il a été témoin et ciblé par plusieurs raids aériens allemands en 1918, marquant ainsi l'histoire locale avec des souvenirs qui perdurent aujourd’hui encore à travers ces lieux de mémoire.

Renseignements supplementaires :
- No 33 abrite le Campus Picpus de la formation médicale au sein du site APHP (Assistance Publique - Hôpitaux de Paris), ancien lieu d'entrée principale pour l’Hôpital Rothschild. Coulorévert, ce bâtiment est un témoignage historique des pratiques hospitalières avant les réformes structurelles et organisationnelles dans le secteur médical français du 20ème siècle à Paris (source : http://www.campus-picpus.ap-hopitalinfo).
  
Références:
1) "Le boulevard de Picpus - Histoire & patrimoine" – Association Amis du Trône et des barrières, consulté le 27

KeyboardInterrupt: 